# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
#import scipy.stats as st

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [2]:
# Save config information
url = "http://api.openweathermap.org/data/2.5/weather?"

city_name = []
city_number = []
latitude = []
temperature = []
humidity = []
cloudiness = []
wind_speed = []



## Generate Cities List

In [3]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []



# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
#below is so I do not go over the rate limits
#cities = ['Kovūr','Mar del Plata','Barrow']
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)


# Print the city count to confirm sufficient count
len(cities)



628

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [4]:
for city in cities[:5]:
    # Build query URL
    query_url = url + "appid=" + weather_api_key + "&q=" + city
    
    try:
        
        #get data
        weather_response = requests.get(query_url)
        weather_json = weather_response.json()
        print (weather_json)
        city_number.append(weather_json['id'])
        city_name.append(weather_json['name'])
        latitude.append(weather_json['coord']['lon'])
        temperature.append(weather_json['main']['temp_max'])
        humidity.append(weather_json['main']['humidity'])
        cloudiness.append(weather_json['clouds']['all'])
        wind_speed.append(weather_json['wind']['speed'])

    except:
        
        print("city not found")





http://api.openweathermap.org/data/2.5/weather?appid=2ce1e88f7291f8fa35b9e5b1e2d5a34c&q=san quintin
{'coord': {'lon': -115.95, 'lat': 30.4833}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 286.31, 'feels_like': 283.42, 'temp_min': 286.31, 'temp_max': 286.31, 'pressure': 1016, 'humidity': 76, 'sea_level': 1016, 'grnd_level': 1015}, 'visibility': 10000, 'wind': {'speed': 3.83, 'deg': 321}, 'clouds': {'all': 99}, 'dt': 1612930817, 'sys': {'country': 'MX', 'sunrise': 1612880946, 'sunset': 1612920425}, 'timezone': -28800, 'id': 3984997, 'name': 'San Quintín', 'cod': 200}
http://api.openweathermap.org/data/2.5/weather?appid=2ce1e88f7291f8fa35b9e5b1e2d5a34c&q=cayenne
{'coord': {'lon': -52.3333, 'lat': 4.9333}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02n'}], 'base': 'stations', 'main': {'temp': 298.15, 'feels_like': 300.48, 'temp_min': 298.15, 'temp_max': 298.15, 'pressu

In [ ]:
print(city_name, city_number, latitude, temperature, humidity, cloudiness, wind_speed)

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
city_results = pd.DataFrame({"City Name" : city_name,
                              "City Number": city_number,
                              "Latitude" : latitude,
                              "Temperature" : temperature,
                              "Humidity" : humidity,
                              "Cloudiness" : cloudiness,
                              "Wind Speed" : wind_speed})

city_results.to_csv("City Results", encoding = "utf-8", index = False)

In [ ]:
city_results

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#how to get the indices?  .iloc
#  Get the indices of cities that have humidity over 100%.
humid_cities = []
####update to 100%
#get a list of cities that have humidity < 100
humid_cities = city_results[city_results["Humidity"] > 100].index

print(humid_cities)


In [ ]:
clean_city_data = city_results.drop(humid_cities,inplace = False)

clean_city_data

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
Latitude = clean_city_data["Latitude"]
Temperature = clean_city_data["Temperature"]
##? when to use plt.scatter and when to use df.plot?

#using matplotlib
plt.scatter(Latitude, Temperature, marker="o", facecolors="red", edgecolors="black")
plt.legend(["test"])
plt.xlabel('Latitued')
plt.ylabel('Temperature')
plt.title(('Latitued vs Temperature'))
#            #s=x_axis, alpha=0.75)


plt.show()

## Latitude vs. Humidity Plot

In [ ]:
Latitude = clean_city_data["Latitude"]
Humidity = clean_city_data["Humidity"]
##?how do I decide which goes first - latitued or humidity 
plt.scatter(Latitude, Humidity, marker="o", facecolors="red", edgecolors="black")
            #s=x_axis, alpha=0.75)
    
plt.show

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

In [ ]:
#the starter code had from scipy.stats import linregress

w_slope, w_intercept, w_cap_r, w_cap_p, w_cap_std_err = linregress(Latitude, Humidity)


print(w_slope)

In [ ]:
#function
# Add the linear regression equation and line to plot
def find_lin_reg(x_values, y_values, title, cordinates):
#x_values = housing_data['RM']
#y_values = housing_data['MEDV']
    (slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
    regress_values = x_values * slope + intercept
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
    plt.scatter(x_values,y_values)
    plt.plot(x_values,regress_values,"r-")
    plt.annotate(line_eq,cordinates,fontsize=15,color="red")
    plt.xlabel('Latitude')
    plt.ylabel(title)
    plt.show()

In [ ]:
northern_data = clean_city_data.loc[clean_city_data["Latitude"]>= 0]
southern_data = clean_city_data.loc[clean_city_data["Latitude"]< 0]
southern_data

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
x_values = northern_data["Latitude"]
y_values = northern_data["Temperature"]

find_lin_reg(x_values, y_values,'Max Temp', (10,265))


####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
x_values = southern_data["Latitude"]
y_values = southern_data["Temperature"]

find_lin_reg(x_values, y_values,'Max Temp', (-160,250))


####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression